In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Current problems!

#Heaviside undefined point, set to 0 at the moment for all of them!
#incorporating the delta functions into M
#'edges' of integrations never include Heavisides!
#still have a c_r and c_al in euler equations!
#organisation of derivatives in euler equations, need to confirm the structure that they will form!


#this will need to be a bit more sophisticated, first order has 3 terms! second order has 7!
#will also need to be able to determine which go with cos and which with sin. also which terms are axissymmetric!
#maybe replace this with an order parameter, then size is determined during the trial function generation!
size = 3

#dont think these are needed!
space_dim = 2
vel_dim = 3

#start by assuming the number of regions and there domians is known, can use the string parsing thing to get them if just a heaviside is passed through!
#this might not be required, can just use len(h)!
regions = 4


#note the r at the front of the string stops pythons default \ behaviour and makes it LATEX
c_r, c_al, c_phi, al, Kn = symbols(r'c_r, c_{\alpha}, c_{\phi}, \alpha, Kn')

#dont think this does anything at all lol!
r = symbols('r', positive=True, real=True, finite=True)
chi, V, psi = symbols(r'\chi, V, \psi')
theta = symbols(r'\theta')





vs = symbols('v1:%d'%(size+1))
bs = symbols('b1:%d'%(size+1))
ls = symbols('l1:%d'%(size+1))
ws = symbols('w1:%d'%(size+1))


#may want to have (r) here -> was required for couette flow
vs = [Function(x)(r) for x in vs]
bs = [Function(x)(r) for x in bs]
ls = [Function(x)(r) for x in ls]
ws = [Function(x)(r) for x in ws]

terms = vs+bs+ls+ws


#c_al is antisym, rest are symmetric

#will want a way to automatically produce this later!
hv = vs[0]*cos(al) + vs[1]*c_r*cos(al) + vs[2]*c_al*sin(al)
hb = bs[0]*cos(al) + bs[1]*c_r*cos(al) + bs[2]*c_al*sin(al)
hl = ls[0]*cos(al) + ls[1]*c_r*cos(al) + ls[2]*c_al*sin(al)
hw = ws[0]*cos(al) + ws[1]*c_r*cos(al) + ws[2]*c_al*sin(al)


h_dom = [[(chi, 0, asin(1/r)), (psi, 0, 2*pi), (V, 0, oo)], 
         [(chi, asin(1/r), pi/2), (psi, 0, 2*pi), (V, 0, oo)], 
         [(chi, pi/2, pi-asin(1/r)), (psi, 0, 2*pi), (V, 0, oo)],
         [(chi, pi-asin(1/r), pi), (psi, 0, 2*pi), (V, 0, oo)]]

#regions are designed for this to be the case!
hn_dom = h_dom[::-1]

h = [hv, hb, hl, hw]
#cannot just choose 0, need to be more careful later!
#h_heavy = (hv*Heaviside(asin(1/r)-chi, 0) + 
#           hb*(Heaviside(chi-asin(1/r), 0) - Heaviside(chi-pi/2, 0)) + 
#           hl*(Heaviside(chi-pi/2, 0) - Heaviside(chi- pi + asin(1/r), 0)) +
#           hw*Heaviside(chi- pi + asin(1/r), 0))

#display(h_heavy)



In [3]:
#substitutes the c's for the spherical coords in terms of V and fancy X


#shit house way of doing this!
for i in range(len(h)):
    h[i] = h[i].subs([(c_r, V*cos(chi)), (c_al, V*sin(chi)*sin(psi)), (c_phi, V*sin(chi)*cos(psi))])
                    
#h_heavy = h_heavy.subs([(c_r, V*cos(chi)), (c_al, V*sin(chi)*sin(psi)), (c_phi, V*sin(chi)*cos(psi))])
#hn_heavy = h_heavy.copy()
#hn_heavy = hn_heavy.subs([(chi, pi-chi), (psi, psi+pi)])


hn = h.copy()

for i in range(len(h)):
    hn[i] = hn[i].subs([(chi, pi-chi), (psi, psi+pi)])
    
#display(h[0])
#display(hn[0])

#heavisides = [Heaviside(asin(1/r)-chi, 0), 
#              Heaviside(chi-asin(1/r), 0) - Heaviside(chi-pi/2, 0), 
#              Heaviside(chi-pi/2, 0) - Heaviside(chi- pi + asin(1/r), 0), 
#              Heaviside(chi- pi + asin(1/r), 0)]

#hn_heavy = (hn[3]*Heaviside(asin(1/r)-chi, 0) + 
#           hn[2]*(Heaviside(chi-asin(1/r), 0) - Heaviside(chi-pi/2, 0)) + 
#           hn[1]*(Heaviside(chi-pi/2, 0) - Heaviside(chi- pi + asin(1/r), 0)) +
#           hn[0]*Heaviside(chi- pi + asin(1/r), 0))

#display(hn_heavy)

In [4]:

#note that this doesn't include anything about the edges of the heaviside!
def disc_integral(kernel, func, domain, regions):
    results = 0
    for i in range(regions):
        #print(i)
        #print(domain[i])
        #don't like this but velocity is always integrated over 3 dims i think??
        #can do this better with the unpack * thing!
        expr = (kernel*func[i]).expand()
        #display(expr)
        #this means the expression is a sum of terms and needs to be split 
        #for efficient integration!
        if type(expr) == Add:
            #print('split')
            results += compute_big_integral(expr, domain[i])#[0], domain[i][1], domain[i][2])
        else:
            results += integrate(kernel*func[i], *domain[i])#[0], domain[i][1], domain[i][2])
        
        
    return results

#ideally use this everytime, needs some way to stop this for single term kernels though, maybe type diff of add vs mul??
#not needed now, implemented above!
def compute_big_discs(kernel, func, domain, regions):
    results = 0
    for i in range(regions):
        #print(i)
        #print(domain[i])
        #don't like this but velocity is always integrated over 3 dims i think??
        #can do this better with the unpack * thing!
        #results += integrate(kernel*func[i], domain[i][0], domain[i][1], domain[i][2])
        results += compute_big_integral(kernel*func[i], domain[i][0], domain[i][1], domain[i][2])
    return results

#integrates each term individually, as sympy doesn't do this for some reason!
def compute_big_integral(to_integrate, domain):
    res = 0
    #display(to_integrate)
    expr = (to_integrate.expand()).args
    
    for i in expr:
        #display(i)
        res += integrate(i, *domain)
        
    return res

#print(hn_dom[0])
#u_r_test = pi**(-3/2)*disc_integral(V*cos(pi-chi)*exp(-V**2)*V**2*sin(chi), hn, hn_dom, regions)
#display(u_r_test.simplify())

In [5]:
#compute L[h(-c)], assume no temperature perturbation, however we do have a density perturbation!

#note currently using hn for these integrations but not sure why!
#should be easy to change now though!
#display(V*cos(pi-chi)*exp(-V**2)*V**2*sin(chi)*h[0])
u_r = pi**(-3/2)*disc_integral(V*cos(pi-chi)*exp(-V**2)*V**2*sin(chi), h, h_dom, regions)

#display(u_r)
u_al = pi**(-3/2)*disc_integral(V*sin(pi-chi)*sin(psi+pi)*exp(-V**2)*V**2*sin(chi), h, h_dom, regions)

#display(u_al)


sigma = pi**(-3/2)*disc_integral(exp(-V**2)*V**2*sin(chi), h, h_dom, regions)


#display(sigma)

In [6]:
#testing stuff done here!

#doesn't lool like integrating heavyside fns works very well
#might still be able to extract the integration limits from the heavysides????
#could also try this as a piecewise func instead??

#still doesn't look great for heaviside, but also heaviside needs to be changed in some way when going from h to h(-c)!!!
#u_r_heavy = pi**(-3/2)*integrate(hn_heavy*V*cos(pi-chi)*exp(-V**2)*V**2*sin(chi), (chi, 0, pi), (psi, 0, 2*pi), (V, 0, oo)).rewrite(Piecewise).doit()
#u_r_heavy = refine(pi**(-3/2)*integrate(Heaviside(chi-asin(1/r), 1)*V*cos(pi-chi)*exp(-V**2)*V**2*sin(chi), (chi, 0, pi), (psi, 0, 2*pi), (V, 0, oo)).rewrite(Piecewise).doit(), Q.positive(r))
#display(refine(u_r_heavy))

display((-sin(chi)).expand().args)
display(type((-sin(chi)).expand()))
if type((-sin(chi)).expand()) == Mul:
    print(True)
display(type((h[0]*sin(chi)).expand()))
#display(((hn_heavy*V*cos(pi-chi)*exp(-V**2)*V**2*sin(chi)).expand()).args[0])

(-1, sin(\chi))

sympy.core.mul.Mul

True


sympy.core.add.Add

In [6]:
#L operator using the negative h and the macro variables found earlier!
L = [(sigma -2*V*cos(chi)*u_r - 2*V*sin(chi)*sin(psi)*u_al - hn[i]) for i in range(regions)]
#display(L[0])
#should have domains matching hn

In [9]:
#[(c_r, V*cos(chi)), (c_al, V*sin(chi)*sin(psi)), (c_phi, V*sin(chi)*cos(psi))]

#only chi derivs are effected by heavisides!
#may be easiest to have a D just with the regions and have a D with the deltas and stuff, as the deltas have to be integrated manually!

#this does NOT include the dirac delta part
#should have same domain as hn!
D = [Kn*(-c_r*hn[i].diff(r) - c_al/r * hn[i].diff(al) + 
     (c_al**2 + c_phi**2)/r*diff_c_r(hn[i]) +  
     (c_phi**2/r * cot(al)- c_r*c_al/r)*diff_c_al(hn[i]) - 
     (c_phi*c_al/r*cot(al) + c_r*c_phi/r)*diff_c_phi(hn[i])) for i in range(regions)]

D = [(D[i].subs([(c_r, V*cos(chi)), (c_al, V*sin(chi)*sin(psi)), (c_phi, V*sin(chi)*cos(psi))])) for i in range(regions)]
#display(D[0])

In [8]:
#NOTE none of these include the derivs of the heaviside yet!

#this should be earlier in the piece!

#will need to include domains and stuff I think!
def diff_c_r(func):
    return cos(chi)*func.diff(V) - sin(chi)/V * func.diff(chi)

    
def diff_c_al(func):
    return sin(psi) * sin(chi)*func.diff(V) + cos(psi)/(V*sin(chi))*func.diff(psi) + sin(psi)*cos(chi)/V * func.diff(chi)
    
    
def diff_c_phi(func):
    return cos(psi) * sin(chi)*func.diff(V) - sin(psi)/(V*sin(chi))*func.diff(psi) + cos(psi)*sin(chi)/V * func.diff(chi)

In [10]:
#need to create the D terms that come from the derivatives of the heavisides!
#display(hn_heavy.diff(chi))
#display(hn_dom[0])
D_dirac = []

diracs = []

#probably not doing this the best way!!!
for i in range(regions):
    d_i_pos = []
    d_i_neg = []

    #exclude the end points!
    #middle 0 is because we only have chi discs!
    if hn_dom[i][0][1] not in [0, pi]:
        d_i_pos.append(DiracDelta(chi-hn_dom[i][0][1]))
        
    #right order for negs
    #this negative causes issues though as the args of this are not simple like for the other one!
    #might be able to do a Mul vs Add type thing again!
    if hn_dom[i][0][2] not in [0, pi]:
        d_i_neg.append(DiracDelta(chi-hn_dom[i][0][2]))
        
    exprs = 0
    #this is terrible!
    for j in d_i_pos:
        exprs += j*hn[i]
    for j in d_i_neg:
        exprs -= j*hn[i]
    
    D_dirac.append(exprs)
    
    #does it matter that the negative is not stored here???
    #diracs.append([d_i_pos, d_i_neg].flatten())
    
    #this is also terrible but works based on the other shit!
    #dont think this will scale well, but probably doesn't matter much as regions wont get very big?
    diracs.append([d for sublist in [d_i_pos, d_i_neg] for d in sublist])
    #diracs.append()
    
#display(diracs)
#display(D_dirac[1])

#for i in D_dirac:
    #display(i)


#dont think i want it to do the integration, need to have the choice of half space dirac integration!
#display(integrate(D_dirac, (chi, 0, pi), (psi, 0, pi/4)))

In [11]:
#this 'integrates' the delta terms in M, start by choosing the half space dirac integral as 1/2

#needs to be more sophisticated!
delta_half = 1/2

M_dirac_kernel = []
for i in range(regions):
    
    #each region has 1 or 2 diracs!
    expr = 0
    for j in diracs[i]:
        #display(solve(j.args[0], chi)[0])
        delta_val = solve(j.args[0], chi)[0]
        #display(delta_val)
        #display(hn[i])
        #this is very not clear atm! the kernel thing that is usually the same for each region 
        #is now different for each thing
        #extra sin(chi) term comes from the kernel-> but chi needs to be replaced for this to work!
        expr += delta_half*((h[i]*(D[-1-i] - L[-1-i] -2*theta*hn[-1-i]))*sin(chi)).subs([(chi, delta_val)])
        #display(expr)
    M_dirac_kernel.append(expr)
    
#display(M_dirac_kernel)
#display(hn_dom)
post_dirac_dom = [x[1:] for x in hn_dom]
#display(post_dirac_dom)
#exp(-V**2)*V**2*r**2*sin(al)

M_dirac = disc_integral(exp(-V**2)*V**2*r**2*sin(al), M_dirac_kernel, post_dirac_dom, regions)
#think this works!
#display(M_dirac)
    

In [184]:
#assume we are given the domains of the regions, but this could be useful to make it more general!
display(srepr(hn_heavy.args[0].args[1]))
display(hn_heavy.args[0])




"Add(Mul(Integer(-1), Symbol('V'), Function('l2')(Symbol('r')), cos(Symbol('\\\\alpha')), cos(Symbol('\\\\chi'))), Mul(Integer(-1), Symbol('V'), Function('l3')(Symbol('r')), sin(Symbol('\\\\alpha')), sin(Symbol('\\\\chi')), sin(Symbol('\\\\psi'))), Mul(Function('l1')(Symbol('r')), cos(Symbol('\\\\alpha'))))"

(-Heaviside(\chi - pi/2, 0) + Heaviside(\chi - asin(1/r), 0))*(-V*l2(r)*cos(\alpha)*cos(\chi) - V*l3(r)*sin(\alpha)*sin(\chi)*sin(\psi) + l1(r)*cos(\alpha))

In [12]:
#time to construct M!!!!
#just try to get the complex euler equations for now, have to check how that goes with Couette flow!



M_kernel = []

for i in range(regions):
    #should be looping backwards through L and D, might need to generalise if opposite regions are not opposite in list!
    #might want to make a function for cumputing M since it is used twice!
    M_kernel.append((h[i]*(D[-1-i] - L[-1-i] -2*theta*hn[-1-i])))

#display(L[-1])
#display(M_kernel[0])

#for i in M_kernel:
#    display(latex(i))

#inconsistant use of word kernel!!!!
M_no_delta = disc_integral(exp(-V**2)*V**2*sin(chi)*r**2*sin(al), M_kernel, h_dom, regions)
#display(M_no_delta)


In [13]:
#still need to add the delta part first!!
#gotta fix these names by golly!
M = M_dirac+M_no_delta
#display(terms)
ee = euler_equations(M, terms, (r))
#display(ee[0])
#this works exactly how we want!
#can loop through the terms etc and get the coeffs for everything!
#a = Poly(ee[0].lhs, vs[0](r))

#display(a.coeffs()[0])
#display(ee[1])
#display(ee[2])
#display(ee[3])

In [17]:
diff_terms = [Derivative(x) for x in terms]

diff_in_ee = []


#determines which derivatives are in which euler equations
for i in ee:
    diff_in_ee.append(i.atoms(Derivative))
#display(diff_terms)

#seems to put the ee's in reverse order, not sure if that can be trusted tbh
#will assume this for now, but have to make sure it generalises properly!
ee = ee[::-1]
diff_in_ee = diff_in_ee[::-1]
#display(diff_in_ee)

solve_ee = []

#should almost be done recursivly!
#this doesn't take forever now but probs still to slow!
for i in range(int(len(ee)/size)):
    temp_eqs = []
    for j in range(size):
        eqn = ee[i+j]
        #display(eqn)
        #display(list(diff_in_ee[i])[j+1])
        temp_eqs.append(solve(eqn, list(diff_in_ee[i])[j], exclude=[Kn, al, theta, c_r, c_al, r], minimal=True, manual=True, implicit=True, check=False))
    
    #display(temp_eqs)
        
        


#fin_eqs = solve(ee, diff_terms, quick=True)
#display(fin_eqs[0])

IndexError: list index out of range

In [16]:
display(temp_eqs[0][0])

0.000994718394324346*(890.932479493073*Kn*r**7*Derivative(w2(r), r) - 890.932479493073*Kn*r**6*sqrt(r**2 - 1.0)*Derivative(w2(r), r) + 89.0932479493073*Kn*r**6*w2(r) - 467.739551733863*Kn*r**6*w3(r) + 445.466239746537*Kn*r**6*w3(r)/cos(\alpha)**2 - 89.0932479493073*Kn*r**5*sqrt(r**2 - 1.0)*w2(r) + 467.739551733863*Kn*r**5*sqrt(r**2 - 1.0)*w3(r) - 445.466239746537*Kn*r**5*sqrt(r**2 - 1.0)*w3(r)/cos(\alpha)**2 - 1336.39871923961*Kn*r**5*sqrt(r**2 - 1.0)*Derivative(w2(r), r) - 502.654824574367*Kn*r**5*w1(r) - 1781.86495898615*Kn*r**5*Derivative(w2(r), r) - 668.199359619805*Kn*r**4*sqrt(r**2 - 1.0)*w2(r) - 44.5466239746537*Kn*r**4*w2(r) + 233.869775866932*Kn*r**4*w3(r) - 222.733119873268*Kn*r**4*w3(r)/cos(\alpha)**2 + 1336.39871923961*Kn*r**3*sqrt(r**2 - 1.0)*Derivative(w2(r), r) + 890.932479493073*Kn*r**3*Derivative(w2(r), r) - 334.099679809902*Kn*r**2*sqrt(r**2 - 1.0)*w2(r) + 417.624599762378*Kn*r**2*sqrt(r**2 - 1.0)*w3(r) - 334.099679809902*Kn*r**2*sqrt(r**2 - 1.0)*w3(r)/cos(\alpha)**2 